In [11]:
import music_trees as mt
import glob

# let's look at which results we have available
result_csvs = glob.glob(str(mt.RESULTS_DIR / 'mdb'/ '*.csv'))
result_csvs

['/home/hugo/lab/music-trees/results/mdb/dmdb-h2a2protonet-v0.csv',
 '/home/hugo/lab/music-trees/results/mdb/dmdb-h2protonet-v1.csv',
 '/home/hugo/lab/music-trees/results/mdb/dmdb-h2a0.5protonet-v0.csv',
 '/home/hugo/lab/music-trees/results/mdb/dmdb-h1protonet-v1.csv',
 '/home/hugo/lab/music-trees/results/mdb/dmdb-baseline-v0.csv']

In [12]:
import pandas as pd
dfs = [pd.read_csv(p) for p in result_csvs]
df = pd.concat(dfs, ignore_index=True)
df

,Unnamed: 0,episode_idx,metric,value,tag,model,n_shot,n_class,logger,checkpoint_callback,...,dataset,batch_size,num_workers,n_query,model_name,learning_rate,d_root,height,taxonomy_name,loss_decay
0,0,0,f1_micro,0.868519,protonet,dmdb-h2a2protonet_v0,32,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,2,deep-mdb,2.0
1,1,0,f1_macro,0.859012,protonet,dmdb-h2a2protonet_v0,32,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,2,deep-mdb,2.0
2,2,0,hlca-mistake,2.154930,protonet,dmdb-h2a2protonet_v0,32,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,2,deep-mdb,2.0
3,3,0,f1_micro,0.868981,prototree-h2,dmdb-h2a2protonet_v0,32,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,2,deep-mdb,2.0
4,4,0,f1_macro,0.806650,prototree-h2,dmdb-h2a2protonet_v0,32,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,2,deep-mdb,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,695,99,hlca-mistake,2.606092,protonet,dmdb-baseline_v0,1,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,0,deep-mdb,-0.5
20996,696,99,f1_micro,0.587500,prototree-h2,dmdb-baseline_v0,1,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,0,deep-mdb,-0.5
20997,697,99,f1_macro,0.561108,prototree-h2,dmdb-baseline_v0,1,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,0,deep-mdb,-0.5
20998,698,99,f1_micro,0.681019,prototree-h1,dmdb-baseline_v0,1,12,True,True,...,mdb,1,20,12,hprotonet,0.03,128,0,deep-mdb,-0.5


# Results Table

In [13]:
import numpy as np
metrics = df['metric'].unique()
tags = df['tag'].unique()
shots = list(reversed(df['n_shot'].unique()))
models = list(reversed(df['model'].unique()))

results = {}
for tag in tags:
    results[tag] = []
    for n_shot in shots:
        for model in models:
            if model == 'hproto_v2': continue
                
            ms = {
                    'model': model,
                    'n_shot': n_shot, 
            }
            for metric in metrics:
                values = np.array(df[(df['metric'] == metric) & \
                            (df['tag'] == tag) & \
                            (df['n_shot'] == n_shot) & \
                            (df['model'] == model) 
                           ]['value'].values)
                
                ms[metric] = f'{values.mean():.4f} ± {values.std():.4f}'
            
            results[tag].append(ms)
            

print(f"{pd.DataFrame(results['protonet'], index=None).to_markdown()}")

/home/hugo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning:

Mean of empty slice.

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning:

invalid value encountered in true_divide

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning:

invalid value encountered in double_scalars

/home/hugo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning:

Mean of empty slice.

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning:

Degrees of freedom <= 0 for slice

/

/home/hugo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning:

Mean of empty slice.

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning:

invalid value encountered in true_divide

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning:

invalid value encountered in double_scalars

/home/hugo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning:

Mean of empty slice.

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning:

Degrees of freedom <= 0 for slice

/

/home/hugo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning:

Mean of empty slice.

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning:

invalid value encountered in true_divide

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning:

invalid value encountered in double_scalars

/home/hugo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning:

Mean of empty slice.

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning:

Degrees of freedom <= 0 for slice

/

|    | model                  |   n_shot | f1_micro        | f1_macro        | hlca-mistake    |
|---:|:-----------------------|---------:|:----------------|:----------------|:----------------|
|  0 | dmdb-baseline_v0       |        1 | 0.5773 ± 0.0825 | 0.5560 ± 0.0854 | 2.4606 ± 0.1984 |
|  1 | dmdb-h1protonet_v1     |        1 | 0.5698 ± 0.0787 | 0.5508 ± 0.0772 | 2.4581 ± 0.1814 |
|  2 | dmdb-h2a0.5protonet_v0 |        1 | 0.5986 ± 0.0888 | 0.5794 ± 0.0934 | 2.4430 ± 0.1846 |
|  3 | dmdb-h2protonet_v1     |        1 | 0.5867 ± 0.0850 | 0.5673 ± 0.0886 | 2.4417 ± 0.1854 |
|  4 | dmdb-h2a2protonet_v0   |        1 | 0.5793 ± 0.0832 | 0.5598 ± 0.0868 | 2.4566 ± 0.1780 |
|  5 | dmdb-baseline_v0       |        2 | 0.6578 ± 0.0685 | 0.6450 ± 0.0720 | 2.4246 ± 0.1816 |
|  6 | dmdb-h1protonet_v1     |        2 | 0.6668 ± 0.0664 | 0.6550 ± 0.0703 | 2.3975 ± 0.1874 |
|  7 | dmdb-h2a0.5protonet_v0 |        2 | 0.6811 ± 0.0772 | 0.6711 ± 0.0806 | 2.3900 ± 0.1961 |
|  8 | dmdb-h2protonet_v1     

/home/hugo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning:

Mean of empty slice.

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning:

invalid value encountered in true_divide

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning:

invalid value encountered in double_scalars

/home/hugo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning:

Mean of empty slice.

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning:

Degrees of freedom <= 0 for slice

/

# Boxplots and Statistical Significance

In [14]:
from scipy.stats import wilcoxon

def significance(df, pairs):
    
    for pair in list(reversed(pairs)):
        (s1, m1), (s2, m2) = pair

        
        df1 = df[(df['n_shot'] == s1) & (df['model'] == m1)].copy()
        df2 = df[(df['n_shot'] == s2) & (df['model'] == m2)].copy()

        stat, p = wilcoxon(df1['value'].values, df2['value'].values)
        
        print(f'\t{pair} --> p < {p}')

In [15]:
import plotly.express as px
import pandas as pd
from itertools import product

metrics = df['metric'].unique()
tags = df['tag'].unique()
shots = df['n_shot'].unique()
models = df['model'].unique()
print(models)

figs = []
for tag in tags:
    for metric in metrics:
        subdf = df
        subdf = df[(df['metric'] == metric) & (df['tag'] == tag)].copy()
        
        # reverse it
        subdf = subdf.astype({'n_shot': 'str'})
        subdf = subdf.iloc[::-1]
        
        pairs = [((str(s1), m1), (str(s2), m2)) for s1, s2, m1, m2 \
                 in product(shots, shots, models, models) \
                    if s1 == s2 and m1 != m2]
        
        print(f'{tag}-{metric}')
#         significance(subdf, pairs)

        if len(subdf) == 0:
            continue
        
        range_y = [0, 1] if 'f1' in metric else None
        
        fig = px.box(subdf, x='n_shot', y='value', color='model',\
                     title=f'{tag}-{metric}', range_y=range_y)
        
        figs.append({
            'tag': tag, 
            'metric': metric, 
            'fig': fig
        })

metrics

['dmdb-h2a2protonet_v0' 'dmdb-h2protonet_v1' 'dmdb-h2a0.5protonet_v0'
 'dmdb-h1protonet_v1' 'dmdb-baseline_v0']
protonet-f1_micro
protonet-f1_macro
protonet-hlca-mistake
prototree-h2-f1_micro
prototree-h2-f1_macro
prototree-h2-hlca-mistake
prototree-h1-f1_micro
prototree-h1-f1_macro
prototree-h1-hlca-mistake


array(['f1_micro', 'f1_macro', 'hlca-mistake'], dtype=object)

In [16]:
figs[0]['fig']

In [17]:
figs[1]['fig']

In [18]:
figs[2]['fig']

In [19]:
figs[3]['fig']

In [20]:
figs[4]['fig']